In [15]:
import pulp
import pandas as pd
import numpy as np
from random import shuffle

In [168]:
class ENO(object):
    def __init__(self, year=2010):
        self.year = year
        self.day = None
        self.hr = None
        
        self.TIME_STEPS = None
        self.NO_OF_DAYS = None
        
        self.BMIN = 0.0
        self.BMAX = 40000.0 #Battery capacity
        self.BOPT = 0.6 * self.BMAX #Assuming 60% of battery is the optimal level
        self.HMAX = 3000
        
        self.senergy = None #matrix with harvested energy data for the entire year
        self.fforecast = None #matrix with forecast values for each day
        
        self.batt = None #battery variable
        self.enp = None #enp at end of hr
        self.henergy = None #harvested energy variable
        self.fcast = None #forecast variable
    
    #function to map total day energy into day_state
    def get_day_state(self,tot_day_energy):
        if (tot_day_energy < 2500):
            day_state = 0
        elif (2500 <= tot_day_energy < 5000):
            day_state = 1
        elif (5000 <= tot_day_energy < 8000):
            day_state = 2
        elif (8000 <= tot_day_energy < 10000):
            day_state = 3
        elif (10000 <= tot_day_energy < 12000):
            day_state = 4
        else:
            day_state = 5
        return int(day_state)

    #function to get the solar data for the given year and prep it
    def get_data(self):
        filename = str(self.year)+'.csv'
        #skiprows=4 to remove unnecessary title texts
        #usecols=4 to read only the Global Solar Radiation (GSR) values
        solar_radiation = pd.read_csv(filename, skiprows=4, encoding='shift_jisx0213', usecols=[4])
        
        #convert dataframe to numpy array
        solar_radiation = solar_radiation.values
        solar_energy = np.array([i *0.0165*1000000*0.15*1000/(60*60) for i in solar_radiation])
        
        #reshape solar_energy into no_of_daysx24 array
        _senergy = solar_energy.reshape(-1,24)
        _senergy[np.isnan(_senergy)] = 0 #convert missing data in CSV files to zero
        self.senergy = _senergy
        
        
        #create a perfect forecaster
        tot_day_energy = np.sum(_senergy, axis=1) #contains total energy harvested on each day
        get_day_state = np.vectorize(self.get_day_state)
        self.fforecast = get_day_state(tot_day_energy)
        
        return 0
    
    def reset(self):
        
        self.get_data() #first get data for the given year
        
        self.TIME_STEPS = self.senergy.shape[1]
        self.NO_OF_DAYS = self.senergy.shape[0]
        
        print("Environment is RESET")
        
        self.day = 0
        self.hr = 0
        
        self.batt = self.BOPT #battery returns to optimal level
        self.enp = self.BOPT - self.batt #enp is reset to zero
        self.henergy = self.senergy[self.day][self.hr] 
        self.fcast = self.fforecast[self.day]
        
        state = [self.batt/self.BMAX, self.enp/(self.BMAX/2), self.henergy/self.HMAX, self.fcast/5] #normalizing all state values within [0,1] interval
        return state
    
    
    #reward function
    def rewardfn(self):
        mu = 0
        sig = 900

        if(np.abs(self.enp) <= 2400): #24hr * 100mW/hr
            return ((1./(np.sqrt(2.*np.pi)*sig)*np.exp(-np.power((self.enp - mu)/sig, 2.)/2)) * 1000000)
        else:
            return -100 - 0.05*np.abs(self.enp)
    
    def step(self, action):
        done = False
        info = "OK"
#         print("Next STEP")
        
        reward = 0
        e_consumed = (action+1)*100
        
        self.batt += (self.henergy - e_consumed)
        self.enp = self.BOPT - self.batt
        
        if(self.hr < self.TIME_STEPS - 1):
            self.hr += 1
            self.henergy = self.senergy[self.day][self.hr] 
        else:
            if(self.day < self.NO_OF_DAYS -1):
                reward = self.rewardfn() #give reward only at the end of the day
                self.hr = 0
                self.day += 1
                self.henergy = self.senergy[self.day][self.hr] 
                self.fcast = self.fforecast[self.day]
            else:
                done = True
                info = "End of the year"
                
        _state = [self.batt/self.BMAX, self.enp/(self.BMAX/2), self.henergy/self.HMAX, self.fcast/5]
        return [_state, reward, done, info]
    

In [169]:
eno=ENO(2010)

In [170]:
s = eno.reset()
print(s)
print("\n")
for i in range(50):
    print([eno.day, eno.hr])
    action = np.random.randint(5)
    print("Action is" , action)
    _s,r,done,info = eno.step(action)
    print([_s,r])
    print("\n")

Environment is RESET
[0.6, 0.0, 0.0, 0.6]


[0, 0]
Action is 3
[[0.59, 0.02, 0.0, 0.6], 0]


[0, 1]
Action is 1
[[0.585, 0.03, 0.0, 0.6], 0]


[0, 2]
Action is 3
[[0.575, 0.05, 0.0, 0.6], 0]


[0, 3]
Action is 0
[[0.5725, 0.055, 0.0, 0.6], 0]


[0, 4]
Action is 2
[[0.565, 0.07, 0.0, 0.6], 0]


[0, 5]
Action is 4
[[0.5525, 0.095, 0.0, 0.6], 0]


[0, 6]
Action is 4
[[0.54, 0.12, 0.059583333333333335, 0.6], 0]


[0, 7]
Action is 3
[[0.53446875, 0.1310625, 0.20625, 0.6], 0]


[0, 8]
Action is 4
[[0.5374375, 0.125125, 0.33458333333333334, 0.6], 0]


[0, 9]
Action is 3
[[0.55253125, 0.0949375, 0.4170833333333333, 0.6], 0]


[0, 10]
Action is 3
[[0.5738125, 0.052375, 0.4560416666666667, 0.6], 0]


[0, 11]
Action is 2
[[0.600515625, -0.00103125, 0.44458333333333333, 0.6], 0]


[0, 12]
Action is 3
[[0.623859375, -0.04771875, 0.37583333333333335, 0.6], 0]


[0, 13]
Action is 3
[[0.642046875, -0.08409375, 0.2635416666666667, 0.6], 0]


[0, 14]
Action is 4
[[0.6493125, -0.098625, 0.126041666666666

In [90]:
eno.step(0)

Next STEP


[[0.5975, 0.005, 0.0, 0.6], 0, False, 'OK']

In [91]:
print([eno.day, eno.hr])

[0, 1]


In [66]:
eno.senergy

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [67]:
eno.forecast

array([3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 0, 2, 3, 2, 3, 3, 2, 3, 2, 1, 2,
       3, 3, 3, 3, 3, 1, 2, 3, 3, 0, 2, 3, 2, 4, 3, 4, 3, 3, 0, 0, 0, 0,
       3, 0, 0, 1, 2, 3, 4, 3, 1, 4, 4, 4, 1, 1, 1, 2, 0, 4, 1, 5, 0, 0,
       2, 0, 3, 5, 5, 5, 5, 2, 3, 4, 4, 4, 5, 4, 5, 2, 0, 0, 2, 5, 2, 2,
       5, 2, 4, 0, 5, 1, 0, 5, 0, 5, 4, 5, 4, 0, 5, 5, 0, 1, 4, 5, 5, 2,
       5, 0, 0, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 3, 0, 4,
       5, 5, 5, 5, 5, 5, 2, 1, 5, 5, 1, 0, 5, 2, 5, 5, 1, 1, 5, 5, 5, 5,
       5, 5, 5, 5, 1, 2, 5, 5, 5, 5, 1, 5, 3, 5, 3, 4, 3, 4, 3, 1, 5, 5,
       2, 3, 3, 2, 4, 2, 5, 2, 4, 3, 3, 2, 5, 2, 5, 2, 2, 1, 4, 4, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 2, 3, 5, 3, 5, 5, 5, 5, 5, 4,
       2, 2, 5, 3, 2, 2, 5, 5, 5, 4, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4,
       5, 5, 5, 5, 5, 4, 5, 5, 0, 3, 5, 5, 4, 3, 3, 2, 0, 5, 5, 4, 2, 5,
       5, 0, 1, 2, 5, 0, 0, 4, 0, 2, 4, 4, 1, 3, 3, 3, 3, 0, 2, 5, 2, 2,
       2, 1, 3, 2, 3, 1, 1, 0, 1, 3, 1, 1, 0, 3, 0,

In [68]:
eno.forecast.shape

(365,)

In [140]:
print()

1
